In [ ]:
import sys
sys.path.append("..")

from pathlib import Path


from src.utils.pb.helpers import TiktokCollectionInfo, VideosCollectionInfo, MetadataCollectionInfo
from src.utils.pb.typehints import TiktokCollectionRecord
from src.utils.pb.collections import MetadataCollection, VideoCollection, Compilation
from src.utils.pb.classes import SingletonPocketBase
from src.compilation.models import VideoCompilation
from src.utils.pb.queries import most_viewed_tiktoks_from_channel

from src.compilation.compile import create_compilation

from src.config import Config

from src.apis.tiktok.api import TiktokAPI, ChannelDetailsAPI

from src.download.helpers import create_download_dir_for_channel
from src.compilation.thumbnails import generate_thumbnails, save_thumbnails_to_path
from src.compilation.helpers import construct_metadata_from_videos, add_likes_views_or_channel_name_to_video

from src.logger import SingletonLogger
from src.config import Config

from src.utils.pb.helpers import get_video_id_from_video_file


from src.utils.pb.actions import fetch_and_insert_videos_from_tiktok_channel

from src.download.actions import download_tiktok_videos_from_same_channel_and_update_pb


logger = SingletonLogger()
pb = SingletonPocketBase()

In [ ]:
def create_video_compilation(title, tiktoks: list[TiktokCollectionRecord]):
    videos = [VideoCollection.validate_record(
        f"{VideosCollectionInfo.Fields.TiktokForeignKey}.{TiktokCollectionInfo.Fields.VideoId}", 
        getattr(tiktok, TiktokCollectionInfo.Fields.VideoId), 
        True) for tiktok in tiktoks]

    videos_for_compilation = [] 
    for video in videos:
        metadata_record = MetadataCollection.validate_record(
            getattr(video, VideosCollectionInfo.Fields.TiktokForeignKey), 
            True
        )
        args = {
            "video": mp.VideoFileClip(Path(getattr(video, VideosCollectionInfo.Fields.VideoPath))), 
            "views_details": (True, getattr(metadata_record, MetadataCollectionInfo.Fields.Views)),
            "likes_details": (True, getattr(metadata_record, MetadataCollectionInfo.Fields.Likes))
        }
        altered_video = add_likes_views_or_channel_name_to_video(**args)
        # altered_video = [clip for clip in altered_video.clips if isinstance(clip, mp.VideoFileClip)][0]
        videos_for_compilation.append(altered_video)
    
    # We use the videos_for_compilation to construct the metadata as we may have additional things onto the video things on top,
    # just as intros or whatever... so use this to be sure 
    compilation_metadata = construct_metadata_from_videos(videos_for_compilation)    

    # Use the selected videos as they hold the actual video location 
    compilation_video = create_compilation(title, videos_for_compilation)

    record = VideoCompilation(title, 
        [Path(getattr(video, VideosCollectionInfo.Fields.VideoPath)) for video in videos], 
        [str(get_video_id_from_video_file(Path(getattr(video, VideosCollectionInfo.Fields.VideoPath)))) for video in videos],
        compilation_video,
        compilation_metadata
    )
    Compilation.create_record(record)
    save_thumbnails_to_path(generate_thumbnails(record.title, record.video_path), compilation_video.parent)

In [ ]:
failed_channels = []
for channel_name in ["mrbeast"]:
    user_details = ChannelDetailsAPI(channel_name, Config.Apis.Tiktok.Cookie)

    tiktok_api = TiktokAPI()
    try:
        await fetch_and_insert_videos_from_tiktok_channel(channel_name, user_details)
    except Exception:
        failed_channels.append(channel_name)
        continue 

    channel_download_dir = create_download_dir_for_channel(channel_name)

    number_of_videos_to_use: int = 5
    most_viewed_videos = most_viewed_tiktoks_from_channel(channel_name, number_of_videos_to_use)
    videos, failed_download = download_tiktok_videos_from_same_channel_and_update_pb(
        most_viewed_videos,
        channel_download_dir
    )

    if len(videos) != number_of_videos_to_use:
        continue 

    create_video_compilation(f"Top {number_of_videos_to_use} most viewed {channel_name} videos!", most_viewed_videos)
    
print(failed_channels)